In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse

## Argument Declare

In [2]:
DATASET_ROOT = "/media/arg_ws3/5E703E3A703E18EB/data/argbot"
DATASET_NAME = "argbot"
cfg = argbot
BASE_NET = "./weights/vgg16_reducedfc.pth"
DATA_DETECTION = ARGBOTDetection
BATCH_SIZE = 32
PRETRAINED_MODEL = "/home/arg_ws3/ssd.pytorch/weights/person/person_67500.pth"
PRETRAINED_ITER = 0
SAVE_MODEL_ITER = 2000
START_ITER = 0
NUM_WORKERS = 4
CUDA = True
LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
GAMMA = 0.1
VISDOM = False
SAVE_FOLDER = "./weights/" + DATASET_NAME + "/"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [3]:
if torch.cuda.is_available():
    if not CUDA:
        print("WTF are u wasting your CUDA device?")
    else:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

## Define function

In [4]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

# Initial model weights & bias
def xavier(param):
    init.xavier_uniform(param)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

# Adjust learning rate during training
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = LR * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        print("Change learning rate to: ", lr)

## Create dataset

In [5]:
dataset = DATA_DETECTION(root=DATASET_ROOT, image_sets=['palm_train'],transform=SSDAugmentation(cfg['min_dim'], MEANS))

classes = dataset.target_transform.class_to_ind
print("Class to index: \n", classes)
classes = sorted(classes.items(), key=lambda kv: kv[1])
label = []
for i in classes:
    label.append(i[0])
label.append('None')
print(label)
true_label = ['person', 'palm']

Class to index: 
 {'person': 0, 'palm': 1}
['person', 'palm', 'None']


## Create model

In [6]:
ssd_pretrained = build_ssd('train', cfg['min_dim'], 2)
if CUDA:
    net = torch.nn.DataParallel(ssd_pretrained)
    cudnn.benchmark = True
    
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
    ssd_pretrained.load_weights(PRETRAINED_MODEL)

Resuming training, loading /home/arg_ws3/ssd.pytorch/weights/person/person_67500.pth...
Loading weights into state dict...
Finished!


/home/arg_ws3/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/arg_ws3/ssd.pytorch/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [7]:
# Delcare SSD Network
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net
if CUDA:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
SAME_CLASS = False
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    if SAME_CLASS:
        print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
        ssd_net.load_weights(PRETRAINED_MODEL)
    else:
        print('Load pretrained model with different classes')
        ssd_pretrained = build_ssd('train', cfg['min_dim'], 2)
        ssd_pretrained.load_weights(PRETRAINED_MODEL)
        ssd_net.vgg = ssd_pretrained.vgg
        ssd_net.extras = ssd_pretrained.extras
        ssd_net.loc = ssd_pretrained.loc
        ssd_net.conf.apply(weights_init)
else:
    print('Initializing weights...')
    vgg_weights = torch.load(BASE_NET) # load vgg pretrained model
    ssd_net.vgg.load_state_dict(vgg_weights)
    ssd_net.extras.apply(weights_init) # Initial SSD model weights & bias
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM,
                weight_decay=WEIGHT_DECAY)
criterion = MultiBoxLoss(BATCH_SIZE ,cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                False, CUDA)

Load pretrained model with different classes
Loading weights into state dict...
Finished!


/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [8]:
#for i in range(len(list(ssd_net.parameters()))-30):
#    list(ssd_net.parameters())[i].data = list(ssd_pretrained.parameters())[i]
#list(ssd_net.conf)[-1] = list(ssd_pretrained.conf)[-1]

## Training

### Initialize

In [9]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')
epoch_size = len(dataset) // BATCH_SIZE
print('Training SSD on:', DATASET_NAME)

data_loader = data.DataLoader(dataset, BATCH_SIZE,
                                num_workers=NUM_WORKERS,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)
batch_iterator = iter(data_loader)

Loading the dataset...
Training SSD on: argbot


### Start training

In [10]:
step_index = 0
for iteration in range(START_ITER, cfg['max_iter']):
    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, GAMMA, step_index)
    
    # make sure data iter not out of range
    try:
        images, targets = next(batch_iterator)
        #print(targets[0][0][4].item(), label[int(targets[0][0][4].item())])
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)
    if CUDA:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    
    # Forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.item()
    conf_loss += loss_c.item()
    
    if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(PRETRAINED_ITER + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')
    
    if iteration != 0 and iteration % SAVE_MODEL_ITER == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                       repr(PRETRAINED_ITER + iteration) + '.pth')
# Save final model
torch.save(ssd_net.state_dict(),
            SAVE_FOLDER + DATASET_NAME + '.pth')

/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()
/home/arg_ws3/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 15.0403 sec.
iter 0 || Loss: 16.8241 ||timer: 0.3054 sec.
iter 10 || Loss: 5.9475 ||timer: 0.3194 sec.
iter 20 || Loss: 4.3332 ||timer: 0.3398 sec.
iter 30 || Loss: 3.4522 ||timer: 0.3338 sec.
iter 40 || Loss: 2.9681 ||timer: 0.0898 sec.
iter 50 || Loss: 3.0218 ||timer: 0.3333 sec.
iter 60 || Loss: 2.2986 ||timer: 0.3270 sec.
iter 70 || Loss: 2.2241 ||timer: 0.3326 sec.
iter 80 || Loss: 2.1337 ||timer: 0.3308 sec.
iter 90 || Loss: 2.0561 ||timer: 0.3326 sec.
iter 100 || Loss: 1.7183 ||timer: 0.3322 sec.
iter 110 || Loss: 1.9025 ||timer: 0.3310 sec.
iter 120 || Loss: 1.7208 ||timer: 0.3352 sec.
iter 130 || Loss: 1.7445 ||timer: 0.3339 sec.
iter 140 || Loss: 1.6521 ||timer: 0.3329 sec.
iter 150 || Loss: 1.4774 ||timer: 0.3362 sec.
iter 160 || Loss: 1.4377 ||timer: 0.3674 sec.
iter 170 || Loss: 1.5587 ||timer: 0.3336 sec.
iter 180 || Loss: 1.7379 ||timer: 0.3265 sec.
iter 190 || Loss: 1.5202 ||timer: 0.3336 sec.
iter 200 || Loss: 1.4680 ||timer: 0.3328 sec.
iter 210 || Loss: 1.5864

iter 1760 || Loss: 1.0118 ||timer: 0.3333 sec.
iter 1770 || Loss: 1.0104 ||timer: 0.3341 sec.
iter 1780 || Loss: 1.0174 ||timer: 0.3342 sec.
iter 1790 || Loss: 0.8595 ||timer: 0.3334 sec.
iter 1800 || Loss: 0.8802 ||timer: 0.3373 sec.
iter 1810 || Loss: 0.8471 ||timer: 0.3260 sec.
iter 1820 || Loss: 0.8859 ||timer: 0.3329 sec.
iter 1830 || Loss: 0.8150 ||timer: 0.3358 sec.
iter 1840 || Loss: 0.9792 ||timer: 0.3338 sec.
iter 1850 || Loss: 0.8497 ||timer: 0.3347 sec.
iter 1860 || Loss: 0.8241 ||timer: 0.3690 sec.
iter 1870 || Loss: 0.9661 ||timer: 0.3340 sec.
iter 1880 || Loss: 0.9516 ||timer: 0.3327 sec.
iter 1890 || Loss: 0.7586 ||timer: 0.3333 sec.
iter 1900 || Loss: 0.8241 ||timer: 0.3336 sec.
iter 1910 || Loss: 0.9589 ||timer: 0.0904 sec.
iter 1920 || Loss: 0.8265 ||timer: 0.3349 sec.
iter 1930 || Loss: 0.8509 ||timer: 0.3287 sec.
iter 1940 || Loss: 0.8452 ||timer: 0.3337 sec.
iter 1950 || Loss: 0.8388 ||timer: 0.3393 sec.
iter 1960 || Loss: 0.8408 ||timer: 0.3338 sec.
iter 1970 || 

iter 3500 || Loss: 0.8713 ||timer: 0.3363 sec.
iter 3510 || Loss: 0.8058 ||timer: 0.3318 sec.
iter 3520 || Loss: 0.7563 ||timer: 0.3335 sec.
iter 3530 || Loss: 0.7703 ||timer: 0.3358 sec.
iter 3540 || Loss: 0.9635 ||timer: 0.3330 sec.
iter 3550 || Loss: 0.6679 ||timer: 0.3375 sec.
iter 3560 || Loss: 0.9510 ||timer: 0.3472 sec.
iter 3570 || Loss: 0.6387 ||timer: 0.3338 sec.
iter 3580 || Loss: 0.8771 ||timer: 0.3365 sec.
iter 3590 || Loss: 0.6378 ||timer: 0.3337 sec.
iter 3600 || Loss: 0.8398 ||timer: 0.3339 sec.
iter 3610 || Loss: 0.8266 ||timer: 0.0903 sec.
iter 3620 || Loss: 0.8544 ||timer: 0.3330 sec.
iter 3630 || Loss: 0.7904 ||timer: 0.3352 sec.
iter 3640 || Loss: 0.8340 ||timer: 0.3334 sec.
iter 3650 || Loss: 0.8305 ||timer: 0.3336 sec.
iter 3660 || Loss: 0.8720 ||timer: 0.3333 sec.
iter 3670 || Loss: 0.7085 ||timer: 0.3429 sec.
iter 3680 || Loss: 1.0592 ||timer: 0.3288 sec.
iter 3690 || Loss: 0.6678 ||timer: 0.3347 sec.
iter 3700 || Loss: 0.8832 ||timer: 0.3348 sec.
iter 3710 || 

iter 5240 || Loss: 0.6020 ||timer: 0.3349 sec.
iter 5250 || Loss: 0.8238 ||timer: 0.3343 sec.
iter 5260 || Loss: 0.6673 ||timer: 0.3441 sec.
iter 5270 || Loss: 0.7724 ||timer: 0.3361 sec.
iter 5280 || Loss: 0.7271 ||timer: 0.3367 sec.
iter 5290 || Loss: 0.5904 ||timer: 0.3343 sec.
iter 5300 || Loss: 0.7190 ||timer: 0.3345 sec.
iter 5310 || Loss: 0.7014 ||timer: 0.0909 sec.
iter 5320 || Loss: 0.9097 ||timer: 0.3334 sec.
iter 5330 || Loss: 0.8406 ||timer: 0.3393 sec.
iter 5340 || Loss: 0.7732 ||timer: 0.3329 sec.
iter 5350 || Loss: 0.6527 ||timer: 0.3353 sec.
iter 5360 || Loss: 0.7685 ||timer: 0.3335 sec.
iter 5370 || Loss: 0.7219 ||timer: 0.3343 sec.
iter 5380 || Loss: 0.8841 ||timer: 0.3258 sec.
iter 5390 || Loss: 0.8621 ||timer: 0.3336 sec.
iter 5400 || Loss: 0.8557 ||timer: 0.3360 sec.
iter 5410 || Loss: 0.8065 ||timer: 0.3337 sec.
iter 5420 || Loss: 0.6124 ||timer: 0.3383 sec.
iter 5430 || Loss: 0.6643 ||timer: 0.3464 sec.
iter 5440 || Loss: 0.6962 ||timer: 0.3350 sec.
iter 5450 || 

...

iter 51280 || Loss: 0.2420 ||timer: 0.3292 sec.
iter 51290 || Loss: 0.3164 ||timer: 0.3330 sec.
iter 51300 || Loss: 0.2963 ||timer: 0.3386 sec.
iter 51310 || Loss: 0.2649 ||timer: 0.3337 sec.
iter 51320 || Loss: 0.2959 ||timer: 0.3393 sec.
iter 51330 || Loss: 0.3206 ||timer: 0.3447 sec.
iter 51340 || Loss: 0.2199 ||timer: 0.3326 sec.
iter 51350 || Loss: 0.1860 ||timer: 0.3350 sec.
iter 51360 || Loss: 0.2268 ||timer: 0.3327 sec.
iter 51370 || Loss: 0.2178 ||timer: 0.3419 sec.
iter 51380 || Loss: 0.2974 ||timer: 0.0903 sec.
iter 51390 || Loss: 0.2106 ||timer: 0.3335 sec.
iter 51400 || Loss: 0.2835 ||timer: 0.3348 sec.
iter 51410 || Loss: 0.2764 ||timer: 0.3344 sec.
iter 51420 || Loss: 0.2635 ||timer: 0.3361 sec.
iter 51430 || Loss: 0.2345 ||timer: 0.3325 sec.
iter 51440 || Loss: 0.2065 ||timer: 0.3332 sec.
iter 51450 || Loss: 0.3153 ||timer: 0.3260 sec.
iter 51460 || Loss: 0.2103 ||timer: 0.3337 sec.
iter 51470 || Loss: 0.2715 ||timer: 0.3354 sec.
iter 51480 || Loss: 0.4419 ||timer: 0.33

iter 52980 || Loss: 0.4058 ||timer: 0.3386 sec.
iter 52990 || Loss: 0.2727 ||timer: 0.3416 sec.
iter 53000 || Loss: 0.3233 ||timer: 0.3411 sec.
iter 53010 || Loss: 0.2523 ||timer: 0.3405 sec.
iter 53020 || Loss: 0.3723 ||timer: 0.3429 sec.
iter 53030 || Loss: 0.2660 ||timer: 0.3495 sec.
iter 53040 || Loss: 0.2644 ||timer: 0.3412 sec.
iter 53050 || Loss: 0.3050 ||timer: 0.3455 sec.
iter 53060 || Loss: 0.2565 ||timer: 0.3419 sec.
iter 53070 || Loss: 0.2430 ||timer: 0.3412 sec.
iter 53080 || Loss: 0.2635 ||timer: 0.0931 sec.
iter 53090 || Loss: 0.2072 ||timer: 0.3407 sec.
iter 53100 || Loss: 0.2933 ||timer: 0.3348 sec.
iter 53110 || Loss: 0.2427 ||timer: 0.3414 sec.
iter 53120 || Loss: 0.2675 ||timer: 0.3429 sec.
iter 53130 || Loss: 0.3253 ||timer: 0.3413 sec.
iter 53140 || Loss: 0.2556 ||timer: 0.3411 sec.
iter 53150 || Loss: 0.2811 ||timer: 0.3439 sec.
iter 53160 || Loss: 0.2050 ||timer: 0.3402 sec.
iter 53170 || Loss: 0.2706 ||timer: 0.3417 sec.
iter 53180 || Loss: 0.1917 ||timer: 0.34

iter 54680 || Loss: 0.2551 ||timer: 0.3372 sec.
iter 54690 || Loss: 0.2135 ||timer: 0.3417 sec.
iter 54700 || Loss: 0.2957 ||timer: 0.3429 sec.
iter 54710 || Loss: 0.3650 ||timer: 0.3409 sec.
iter 54720 || Loss: 0.2872 ||timer: 0.3445 sec.
iter 54730 || Loss: 0.3091 ||timer: 0.3558 sec.
iter 54740 || Loss: 0.2669 ||timer: 0.3399 sec.
iter 54750 || Loss: 0.3451 ||timer: 0.3421 sec.
iter 54760 || Loss: 0.2744 ||timer: 0.3414 sec.
iter 54770 || Loss: 0.3196 ||timer: 0.3419 sec.
iter 54780 || Loss: 0.2668 ||timer: 0.0899 sec.
iter 54790 || Loss: 0.2161 ||timer: 0.3404 sec.
iter 54800 || Loss: 0.2950 ||timer: 0.3425 sec.
iter 54810 || Loss: 0.2600 ||timer: 0.3417 sec.
iter 54820 || Loss: 0.2128 ||timer: 0.3433 sec.
iter 54830 || Loss: 0.3439 ||timer: 0.3405 sec.
iter 54840 || Loss: 0.3261 ||timer: 0.3403 sec.
iter 54850 || Loss: 0.2822 ||timer: 0.3413 sec.
iter 54860 || Loss: 0.2416 ||timer: 0.3409 sec.
iter 54870 || Loss: 0.3047 ||timer: 0.3469 sec.
iter 54880 || Loss: 0.2112 ||timer: 0.34

iter 56380 || Loss: 0.3421 ||timer: 0.3396 sec.
iter 56390 || Loss: 0.2035 ||timer: 0.3405 sec.
iter 56400 || Loss: 0.4024 ||timer: 0.3426 sec.
iter 56410 || Loss: 0.2446 ||timer: 0.3411 sec.
iter 56420 || Loss: 0.2579 ||timer: 0.3431 sec.
iter 56430 || Loss: 0.2005 ||timer: 0.3336 sec.
iter 56440 || Loss: 0.2849 ||timer: 0.3402 sec.
iter 56450 || Loss: 0.3079 ||timer: 0.3452 sec.
iter 56460 || Loss: 0.2634 ||timer: 0.3411 sec.
iter 56470 || Loss: 0.2851 ||timer: 0.3431 sec.
iter 56480 || Loss: 0.1909 ||timer: 0.0928 sec.
iter 56490 || Loss: 0.2446 ||timer: 0.3479 sec.
iter 56500 || Loss: 0.3556 ||timer: 0.3437 sec.
iter 56510 || Loss: 0.2538 ||timer: 0.3416 sec.
iter 56520 || Loss: 0.2190 ||timer: 0.3450 sec.
iter 56530 || Loss: 0.2866 ||timer: 0.3406 sec.
iter 56540 || Loss: 0.2686 ||timer: 0.3409 sec.
iter 56550 || Loss: 0.3010 ||timer: 0.3463 sec.
iter 56560 || Loss: 0.1852 ||timer: 0.3408 sec.
iter 56570 || Loss: 0.2872 ||timer: 0.3391 sec.
iter 56580 || Loss: 0.1645 ||timer: 0.33

KeyboardInterrupt: 

In [11]:
torch.cuda.empty_cache()